In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from sponge_bob_magic.session_handler import State

spark = State().session
spark

In [6]:
from rs_datasets import MovieLens

data = MovieLens("1m")
data.info()

ratings


,user_id,item_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968



users


,user_id,gender,age,occupation,zip_code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117



items


,item_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [9]:
from sponge_bob_magic.data_preparator import DataPreparator

log = DataPreparator().transform(
    data=data.ratings,
    columns_names={
        "user_id": "user_id",
        "item_id": "item_id",
        "relevance": "rating",
        "timestamp": "timestamp"
    }
)

In [10]:
from sponge_bob_magic.splitters import UserSplitter

second_stage_splitter = UserSplitter(
    drop_cold_items=True,
    drop_cold_users=True,
    item_test_size=1,
    user_test_size=1000,
    seed=1234,
    shuffle=True
)

In [11]:
from sponge_bob_magic.models import ALSWrap

first_model = ALSWrap(rank=96)

In [12]:
from sponge_bob_magic.models import ClassifierRec

second_model = ClassifierRec(numTrees=100)

In [13]:
from sponge_bob_magic.scenarios import TwoStagesScenario

two_stages = TwoStagesScenario(
    second_stage_splitter=second_stage_splitter,
    second_model=second_model,
    first_model=first_model
)

In [14]:
recs = two_stages.get_recs(log, 10)

27-Apr-20 08:03:15, sponge_bob_magic, DEBUG: mixed_train stat: total lines: 999209, total users: 6040, total items: 3706
27-Apr-20 08:03:24, sponge_bob_magic, DEBUG: test stat: total lines: 1000, total users: 1000, total items: 663
27-Apr-20 08:03:28, sponge_bob_magic, DEBUG: first_train stat: total lines: 601932, total users: 6040, total items: 3619
27-Apr-20 08:03:31, sponge_bob_magic, DEBUG: first_test stat: total lines: 397277, total users: 6040, total items: 3613
27-Apr-20 08:03:31, sponge_bob_magic, DEBUG: Предварительная стадия обучения (pre-fit)
27-Apr-20 08:03:33, sponge_bob_magic, DEBUG: Основная стадия обучения (fit)
27-Apr-20 08:03:33, sponge_bob_magic, DEBUG: Индексирование данных
27-Apr-20 08:03:33, sponge_bob_magic, DEBUG: Обучение модели
27-Apr-20 08:04:11, sponge_bob_magic, DEBUG: баланс классов: положительных 157562 из 604000
27-Apr-20 08:04:12, sponge_bob_magic, DEBUG: Предварительная стадия обучения (pre-fit)
27-Apr-20 08:04:18, sponge_bob_magic, DEBUG: Основная ста

In [15]:
train, test = second_stage_splitter.split(log)

In [16]:
first_recs = first_model.fit_predict(
    log=train,
    k=10,
    users=test.select("user_id").distinct().cache(),
    items=test.select("item_id").distinct().cache(),
)

27-Apr-20 08:06:52, sponge_bob_magic, DEBUG: Предварительная стадия обучения (pre-fit)
27-Apr-20 08:06:53, sponge_bob_magic, DEBUG: Основная стадия обучения (fit)
27-Apr-20 08:06:53, sponge_bob_magic, DEBUG: Индексирование данных
27-Apr-20 08:06:53, sponge_bob_magic, DEBUG: Обучение модели


In [17]:
two_stages.experiment.add_result("first_stage", first_recs)

In [18]:
two_stages.experiment.pandas_df

,HitRate@10
two_stages_scenario,0.002
first_stage,0.324
